In [103]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
import numpy as np
import torch
import pprint
import matplotlib.pyplot as plt
import sys
import pickle
import argparse
import logging
import os

# from ginkgo import invMass_ginkgo
# from invMass_ginkgo import *
from invMass_ginkgo_node import *
from ginkgo.utils import get_logger


logger = get_logger(level = logging.WARNING)
fh = logging.FileHandler('spam.log')
fh.setLevel(logging.WARNING)
logger.addHandler(fh)


"""
create node class
understand original output
simplify recursion logic further

define typology of a tree
access data on each node

call algo to generate tree
have a fully defined tree with 4D vectors for each node

generate likelihood for each tree 
compute eq (12)
eq(14) simplifies to eq(8), apply 8 repeatedly when matching l and r

"""

'\ncreate node class\nunderstand original output\nsimplify recursion logic further\n\ndefine typology of a tree\naccess data on each node\n\ncall algo to generate tree\nhave a fully defined tree with 4D vectors for each node\n\ngenerate likelihood for each tree \ncompute eq (12)\neq(14) simplifies to eq(8), apply 8 repeatedly when matching l and r\n\n'

In [105]:
"""Parameters"""


rate2 = torch.tensor(8.)

# Parameters to get ~<10 constituents to test the trellis algorithm
pt_min = torch.tensor(4.**2)

### Physics inspired parameters to get ~ between 20 and 50 constituents
W_rate = 3.
QCD_rate = 1.5

QCD_mass = 30.
class ginkgo_simulator():
    def __init__(self,
                 rate,
                 pt_cut,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP,
                 root_rate= 1.5,
                ):
        
        self.root_rate = root_rate
        self.rate = rate
        self.pt_cut = pt_cut
        self.M2start = torch.tensor(M2start) # mass squared to start with
        self.Nsamples = Nsamples
        self.minLeaves = minLeaves
        self.maxLeaves = maxLeaves
        self.maxNTry = maxNTry
        self.jetType = jetType # W or QCD 
        self.jetM = np.sqrt(M2start) # mass to start with
        self.jetdir = np.array([1,1,1])
        self.jetP = jetP
        self.jetvec = self.jetP * self.jetdir / np.linalg.norm(self.jetdir)
        self.jet4vec = np.concatenate(([np.sqrt(self.jetP ** 2 + self.jetM ** 2)], self.jetvec))
        logger.debug(f"jet4vec = {self.jet4vec}")
        
        if jetType == "W":
            # defined in paper, W jets have a different root rate
            self.rate=torch.tensor([self.root_rate,self.rate])
        elif jetType == "QCD":
            # QCD jets maintain the same rate throughout
            self.rate=torch.tensor([self.rate,self.rate])
        else:
            raise ValueError("Choose a valid jet type between W or QCD")



    def simulator(self):

        simulator = Simulator(jet_p = self.jet4vec,
                                         pt_cut = float(self.pt_cut),
                                         Delta_0 = self.M2start,
                                         M_hard = self.jetM ,
                                         num_samples = int(self.Nsamples),
                                         minLeaves = int(self.minLeaves),
                                         maxLeaves = int(self.maxLeaves),
                                         maxNTry = int(self.maxNTry)
                                         )
        return simulator
       
    def generate(self):
        
        simulator = self.simulator()
        jet_list = simulator(self.rate)

        logger.debug(f"---"*10)
        logger.debug(f"jet_list = {jet_list}")
        
        return jet_list

In [106]:
# the range of leaves that you would consider as valid generations
minLeaves = 3
maxLeaves = 100

# number of jets you wish to generate
Nsamples = 1

# exponential rate parameter
rate = 1.5

# mass squared cut off to yield leaves
pt_cut =  torch.tensor(1.1**2)

# mass squared to start with
M2start = 80.**2

# the maximum times you are willing to try to get Nsamples
maxNTry = 1

jetP=400.

In [107]:
jetType ="QCD"


ginkgo = ginkgo_simulator(
                 rate,
                 pt_cut ,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP)

QCD_jets = ginkgo.generate()

Node 0
 Vec4: [407.92156109 230.94010768 230.94010768 230.94010768]
 Decay Rate: 1
 Mass Squared: tensor(6400.)
 Log Likelihood: -17.81020621775468
 DIJ List: -17.81020621775468 1.6218913765689818e-06 0.12336203580467386 358.11161709100537

Node 1
 Vec4: [347.17953319 206.31130828 183.01979919 203.13578627]
 Decay Rate: tensor(0.5014)
 Mass Squared: tensor(3208.8779)
 Log Likelihood: -15.302189141073296
 DIJ List: -15.302189141073296 5.3720053236124093e-08 0.002658854779892041 7.5596234616246205

Node 2
 Vec4: [60.74200844 24.62878839 47.92029748 27.8043104 ]
 Decay Rate: tensor(0.0249)
 Mass Squared: tensor(13.5799)
 Log Likelihood: -5.766896754064628
 DIJ List: -5.766896754064628 6.387756023595487e-07 0.0008013726363122524 0.273191761022304

Node 3
 Vec4: [278.73192579 166.00830416 148.10767879 160.45109816]
 Decay Rate: tensor(0.7642)
 Mass Squared: tensor(2452.2903)
 Log Likelihood: -16.894928246297905
 DIJ List: -16.894928246297905 1.822036449550953e-06 0.059885727743715624 106.87

In [108]:
root_rate = 4.
jetType ="W"


ginkgo= ginkgo_simulator(
                 rate,
                 pt_cut ,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP,
                 root_rate)

W_jets = ginkgo.generate()

Node 0
 Vec4: [407.92156109 230.94010768 230.94010768 230.94010768]
 Decay Rate: 1
 Mass Squared: tensor(6400.)
 Log Likelihood: -16.87916666635572
 DIJ List: -16.87916666635572 2.1311053609217555e-07 0.018215714956400713 21.866065749121173

Node 1
 Vec4: [42.5442308  27.79456832 20.68472053 23.60076737]
 Decay Rate: tensor(0.6441)
 Mass Squared: tensor(52.6197)
 Log Likelihood: -8.590844898977348
 DIJ List: -8.590844898977348 2.7868014861548175e-06 0.0016314668539831854 0.17844156999601773

Node 2
 Vec4: [365.3773157  203.1455311  210.25537889 207.33933204]
 Decay Rate: tensor(0.7868)
 Mass Squared: tensor(5035.5537)
 Log Likelihood: -18.34965446547515
 DIJ List: -18.34965446547515 2.3730454236879086e-06 0.10156558218891365 823.8316379208084

Node 3
 Vec4: [12.87783439  8.57822067  5.98239571  7.31107018]
 Decay Rate: tensor(0.8900)
 Mass Squared: tensor(3.0119)
 Log Likelihood: -3.318405538358756
 DIJ List: -3.318405538358756 0.0008691417852972334 0.056066624506668335 0.3483956007810